In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open('../styles/custom.css', 'r').read()
    return HTML(styles)
css_styling()

# Vortex panel method

The previous notebook introduced the concept of vortex sheets. We will now extend this concept to solve to the flow around general objects by setting up and solving a system of linear equations.

## General vortex sheets

A curved vortex sheet with a variable strength can describe the flow around any immersed object. This is achieved by having the sheet act as an infinitely thin version of the boundary layer to enforce the no-slip boundary condition. 

---

<img src="resources/impulse.png" width="700">

---

In other words we use the sheets to force the tangential velocity $u_s$ to zero at every point $s$ on the body surface $\cal S$

\begin{equation}
u_s = \vec u \cdot \hat s = 0 \quad s \in \cal S
\end{equation}

From the previous notebook, we know the velocity at any point is determined by an integral over the whole vortex sheet. Therefore, the tangential velocity condition is

\begin{equation}
\left[\vec U_\infty+\frac{\partial}{\partial \vec x}\oint_{\cal S} \frac{\gamma(s')}{2\pi}\theta(s,s')\ ds'\right] \cdot\hat s = 0 
\end{equation}

where $\vec U_\infty$ is the background velocity that has been added by superposition. 

**If we can use this equation to determine the strength distribution $\gamma(s)$ along the sheet then we will have solved for the potential flow around the body!**

## Discrete vortex panels

For general body surface shapes the velocity is a highly nonlinear function of $\gamma(s)$, rendering analytic solution unlikely. We could attempt some complex analytic expansions, but why would we want to do that?

##### Numerical fundamental: Discritization
##### Replace continuous functions with linear approximations

We already know that the velocity depends **linearly** on $\gamma$ for a vortex panel. This makes it easy to solve for $\gamma$ as a function of $u_s$. And we can add any number of panels together using superposition.

Therefore, if we break up the continuous sheet into a series of vortex panels, we can approximate the tangential velocity condition above, and use it to solve for $\gamma$. 

---
<img src="resources/graphics2.png" width="700">

---

This is the essence of the *vortex panel method*.

## Linear velocity function

First, lets write the velocity induced by a single panel in a way that makes the linearity explicit:

$$ \vec u(x,y)=\gamma\ \vec f(x,y \ |\ x_0,y_0,x_1,y_1)$$

where $\vec f$ is a function that depends on the panel geometry. 

In fact, we've already written all the code we need to evaluate this function in VortexPanel module, we just need to import it again. 

In [ ]:
import numpy
from matplotlib import pyplot
%matplotlib inline
from VortexPanel import Panel

##### Quiz 1

Which `Panel` function can we use to evaluate `f`?

1. `Panel.plot()`
1. `Panel.velocity(gamma=0)`
1. `Panel.velocity(gamma=1)`

Hint: Use `help` again to see the function arguments if you need to.

## Array of panels

The next step is to extend this to more than one panel.

To help make this more concrete, lets consider a polygon-shape body. I found this beautiful little equation to define [regular polygons](http://math.stackexchange.com/questions/41940/is-there-an-equation-to-describe-regular-polygons/41954#41954).
$$ r=\frac{\cos\left(\frac{\pi}{n}\right)}{\cos\left(\left(\theta \mod \frac{2\pi}{n}\right) -\frac{\pi}{n}\right)} $$

In [ ]:
def polygon(theta,N_sides):
    a = theta % (2.*numpy.pi/N_sides)-numpy.pi/N_sides
    r = numpy.cos(numpy.pi/N_sides)/numpy.cos(a)
    return [r*numpy.cos(theta),r*numpy.sin(theta)]

where `N_sides` is the number of sides in the polygon, and `theta`=$\theta$ is the polar angle around the origin.

Lets define a function to make a polygon-shaped array of panels.

In [ ]:
def make_polygon(N_panels,N_sides):
    # define the end-points
    theta = numpy.linspace(0, -2*numpy.pi, N_panels+1)   # equal radial spacing
    x_ends,y_ends = polygon(theta, N_sides)              # get the coordinates

    # define the panels
    panels = numpy.empty(N_panels, dtype=object)         # empty array of panels
    for i in range(N_panels):                            # fill the array
        panels[i] = Panel(x_ends[i], y_ends[i], x_ends[i+1], y_ends[i+1])

    return panels

Lets plot it to check if we did it right...

In [ ]:
triangle = make_polygon(N_panels=3,N_sides=3)
for p in triangle: p.plot()

Looks good. Note the code 

`for p in triangle:` 

loops through all the panels in the array. Very handy and clean.

---
Now that we have an example, what is the velocity induced by these panels and the uniform flow? 

Using superposition, the total velocity at any point $x,y$ is simply

\begin{equation}
\vec u(x,y) = \vec U_\infty+\sum_{j=0}^{N-1} \gamma_j \ \vec f_j(x,y)
\end{equation}

where we use the index  $j$ to label each of the $N$ panels.

This can be coded as:

In [ ]:
def flow_velocity(panels,x,y,alpha=0):
    # get the uniform velocity ( make it the same size & shape as x )
    u = numpy.cos(alpha)*numpy.ones_like(x)
    v = numpy.sin(alpha)*numpy.ones_like(x)
    
    # add the velocity contribution from each panel
    for p in panels:
        u0,v0 = p.velocity(x,y)
        u = u+u0
        v = v+v0
    
    return u, v

where the `alpha` argument defines the angle of attack. 

To visualize this, I've combined `flow_velocity` along with the grid definition and plotting code from the last notebook to define a `plot_flow` function. Let's import it.

In [ ]:
from VortexPanel import plot_flow
help(plot_flow)

This function takes __any__ array of panels as input, and returns a plot. Easy.

Let's plot the triangle flow...

In [ ]:
plot_flow(triangle)

##### Quiz 2

Why is the flow going through the body above?

1. We set `gamma=0` for the panels
1. We haven't applied the no-slip condition
1. We haven't determined the correct `gamma` for each panel

## System of linear equations

Applying the no-slip boundary condition (1) to the total velocity (3) we have:

$$ u_s = \left[\vec U_\infty + \sum_{j=0}^{N-1} \gamma_j \ \vec f_j(x,y)\right]\cdot\hat s=0 $$

We note again that this equation is **linear** in $\gamma$. 

So the goal is to set $\gamma$ on each panel such that this condition is enforced on the body.

##### Quiz 3

How many unknowns are there?

1. $1$
1. $N$
1. $N^2$

But we only have one equation, the no-slip condition... right?

##### Numerical fundamental: Consistency
##### Develop enough equations to match the unknowns

For a linear system of equations to be consistent, that is for it to have a solution, we need as many equations as unknowns. 

Luckily the no-slip condition is a continuous equation - it applies to *every* point on the body. **Therefore, we can evaluate the boundary equation on each panel**. Then we will have a consistent linear system.

##### Quiz 4

What tangential velocity does panel $i$ induce on the negative (body) side of itself?

1. $u_s = \frac 12 \gamma_i$
1. $u_s = \gamma_i \ \vec f_i(x_i,y_i) \cdot \hat s_i$
1. $u_s = \vec U_\infty \cdot \hat s_i$

where $\hat s_i$ and $(x_i,y_i)$ are the tangent vector and center of panel $i$.

Hint: think about the previous notebook.

---

Using this relation, the tangential velocity at the center of panel $i$ is

$$ \frac 12 \gamma_i + \left[ \vec U_\infty + \sum_{j=0, j\ne i}^N \gamma_j \ \vec f_j(x_i,y_i)\right]\cdot\hat s_i= 0 $$

Let's write the summation as an array inner product to separate out the knowns from the unknowns:

$$
\begin{bmatrix} \vec f_0(x_i,y_i)\cdot\hat s_i & \vec f_1(x_i,y_i)\cdot\hat s_i & \cdots & \frac 12 & \cdots & \vec f_{N-1}(x_i,y_i)\cdot\hat s_i\end{bmatrix} \times \begin{bmatrix} \gamma_0 \\ \gamma_1 \\ \vdots \\ \gamma_i \\ \vdots \\ \gamma_{N-1} \end{bmatrix} + \vec U_\infty \cdot \hat s_i = 0
$$

Written like this, we can see two things:

 - The no-slip condition on panel $i$ depends on the strength at every panel, and
 - This is just the $i$th row of a matrix of equations

\begin{equation}
\begin{bmatrix} 
\frac 12  & \vec f_1(x_0,y_0)\cdot\hat s_0 & \cdots & \vec f_{N-1}(x_0,y_0)\cdot\hat s_0\\[0.5em]
\vec f_0(x_1,y_1)\cdot\hat s_1 & \frac 12 & \cdots & \vec f_{N-1}(x_1,y_1)\cdot\hat s_1 \\[0.5em]
\vdots & \vdots & \ddots & \vdots \\[0.5em] 
\vec f_0(x_{N-1},y_{N-1})\cdot\hat s_{N-1} & \vec f_1(x_{N-1},y_{N-1})\cdot\hat s_{N-1} & \cdots & \frac 12
\end{bmatrix} 
\times \begin{bmatrix} \gamma_0 \\[0.9em] \gamma_1 \\[0.9em] \vdots \\[0.9em] \gamma_{N-1} \end{bmatrix} 
= -\begin{bmatrix} \vec U_\infty\cdot\hat s_0 \\[0.7em] \vec U_\infty\cdot\hat s_1 \\[0.7em] \vdots \\[0.7em] \vec U_\infty\cdot\hat s_{N-1} \end{bmatrix}
\end{equation}

This defines the complete linear system. 

Lets review what we've done. 
1. We started with the analytic integral differential equation (2) which determines the continuous vortex sheet strength $\gamma(s)$.
1. We broke that sheet up into a finite number of linear panels each with their own unknown $\gamma_i$. 
1. We applied the no-slip condition to each panel and used it to assemble a linear system of equations (4). 

Therefore, the solution $\gamma_i$ of (4) is a finite numerical approximation to the continuous analytic solution $\gamma(s)$. 

*This same basic procedure is applied whenever numerically approximating partial differential equations such as the Navier-Stokes.*

---

All well and good having equation (4), but this is a dense linear algebra problem. How in the world will we determine $\gamma_i$? Actully, this is trivial.

##### Numerical fundamental: Linear Algebra Packages
##### Never write your own matrix solver

Every worthwhile numerical language has a set of linear algebra solution routines - in numpy it is the [`linalg` package](http://docs.scipy.org/doc/numpy/reference/routines.linalg.html). I've written the function `solve_gamma` to set-up the system of equations and call `numpy.linalg.solve`. **That is it!** 

Let's import it and use `help` to get the summary:

In [ ]:
from VortexPanel import solve_gamma
help(solve_gamma)

The function takes in __any__ array of Panels and an angle of attack, and sets the strength on each panel such that the no-slip condition is enforced. Note that there is no output returned by the function. It just modified the input panels.

Let's test it out!

In [ ]:
# define geometry
triangle = make_polygon(N_panels=3,N_sides=3)    
solve_gamma(triangle)  # solve for gamma
plot_flow(triangle)    # compute flow field and plot

Much better! But...

##### Quiz 5

Why is there still flow through the wedge?

1. Modeling error       (ie incorrect conditions)
1. Numerical error      (ie insufficient resolution)
1. Implementation error (ie inadequate care)

(Hint: one of these is immediately testable.)

##### Numerical Fundamental: Convergence with resolution
##### The more panels you use, the closer you should get to the analytic solution

## Other shapes

We can now get the flow around **any** shape! Let try a circle. We follow the same pattern as we did with the triangle - defining the end points and then adding each panel to the array. The only difference is that the `ends` are defined using $\cos$ and $\sin$ instead of the `polygon` function.

In [ ]:
def make_circle(N, xcen=0, ycen=0):
    # define the end-points of the panels for a unit circle
    theta = numpy.linspace(0, -2*numpy.pi, N+1)
    x_ends = xcen+numpy.cos(theta)
    y_ends = ycen+numpy.sin(theta)
    
    # define the panels
    circle = numpy.empty(N, dtype=object)
    for i in range(N):
        circle[i] = Panel(x_ends[i], y_ends[i], x_ends[i+1], y_ends[i+1])
    
    return circle

In [ ]:
N = 32
circle = make_circle(N)     # make the shape
gamma = solve_gamma(circle) # solve for gamma
plot_flow(circle)           # compute flow field and plot

Looks about right!

Note that this function takes optional arguments to set the $x,y$ location of the center of the circle. Try them out...


##### Quiz 6

This vortex panel method can be used to solve for the flow around:

1. an ellipse
1. a pair of tandem bodies
1. a rudder

---
##### Your turn #2

 - ** Modify ** the `make_ellipse` function below to generate an ellipse instead of a circle when supplied with an aspect ratio `t_c`=$t/c$.
 - ** Create ** a 2:1 ellipse geometry shifted to be centered at $x=2,y=0$
 - ** Discuss ** whether the maximum speed around the ellipse is greater or less than that around the circle.
 - ** Combine ** the triangle and ellipse geometry together into one set of panels using `numpy.concatenate((body_1,body_2))` and solve for the flow.
 - ** Discuss ** if there is a *wake* between the bodies. Why or why not?

---

 
##### Solution #2

In [ ]:
def make_ellipse(N, xcen=0, ycen=0, t_c=1):
    theta = numpy.linspace(0, -2*numpy.pi, N+1)
    # your code here to define the end-points of the panels for an ellipse
    x_ends = xcen+numpy.cos(theta)  # adjust?
    y_ends = ycen+numpy.sin(theta)  # adjust?
    
    # define the panels
    ellipse = numpy.empty(N, dtype=object)
    for i in range(N):
        ellipse[i] = Panel(x_ends[i], y_ends[i], x_ends[i+1], y_ends[i+1])
    
    return ellipse

#ellipse = make_ellipse(N,t_c=0.5,xcen=2)   # make the shape
#solve_gamma(ellipse)                       # solve for gamma
#plot_flow(ellipse,size=4)                  # compute flow field and plot

In [ ]:
# pair = ?   your code using numpy.concatenate((body_1,body_2))